# The Farmer's Problem
[Murwan Siddig](mailto:msiddig@clemson.edu)

--------------------------------------------------------------------


### Problem statement

* You can grow Wheat, Corn, or Beans on 500 acres.
* You need 200 tons of wheat and 240 tons of corn to feed your cattle.
* You can grow these food on your land or buy from a super farmer.

* You can sell your product at the price of:
    * Wheat: $\$170$ per ton.
    * Corn: $\$150$ per ton.
    * Beans: $\$36$ per ton for the first $6000$ tons, and $\$10$ per ton after that.
* You can buy product from the super farmer at the price of:
    * Wheat: $\$238$ per ton.
    * Corn: $\$210$ per ton.    
* Data summary:
|                              |Wheat|Corn |Beans                        |
|------------------------------|-----|-----|-----------------------------|
|Yield (T/acre)                |$2.5$|$3$  |$20$                         |
|Production Cost (\$/per acre) |$150$|$230$|$260$                        |
|Selling price                 |$170$|$150$|$36/10$ (before/after $6000$)|
|Purchase price                |$238$|$210$|-                            |
|Cattle demand                 |$200$|$240$|-                            |

* Question: Suppose you have 500 acres of land for growing crops, what is your planting plan?

### Deterministic Problem formulation

* **Decision variables**
    * First-stage decisions:
        * $x_1$: acres of wheat planted
        * $x_2$: acres of corn planted
        * $x_3$: acres of beans planted
        
    * Second-stage decisions:
        * $w_1$: tons of wheat sold at regular price
        * $w_2$: tons of corn sold at regular price
        * $w_3$: tons of beans sold at regular price
        * $e$: tons of beans sold at extra price
        * $y_1$: tons of wheat purchased
        * $y_2$: tons of corn purchased

\begin{aligned}
\max_{x,w,y} \quad & 170w_1+150w_2+36w_3+10e-(150x_1+230x_2+260x_3+238y_1+210y_2)\\
\textrm{s.t.} \quad & x_1+x_2+x_3\leq 500\\
  & 2.5x_1+y_1-w_1 = 200\\
  & 3x_2+y_2-w_2 = 240\\
  & 20x_3-w_3-e = 0\\
  & w_3 \leq 6000 \\
\end{aligned}

In [1]:
# Input data
yield=[2.5, 3, 20]; 
cost=[150, 230, 260];
price = [170, 150, 36];
over_price = 10; # Only for beans
over_size = 6000; # Only for beans
extra_price = [238, 210]; # Only for the first two crops
demand = [200, 240];
land_size = 500;
num_crops = 3;
num_purch = 2;

In [2]:
# techincal lines
using JuMP
using Gurobi

#create gurobi environment
const GRB_ENV = Gurobi.Env();

Academic license - for non-commercial use only - expires 2021-05-08


In [3]:
#Define the model
farmer_det = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GRB_ENV), "OutputFlag" => 0));

#Define the variables
@variables(farmer_det,
           begin
              0 <= x[i=1:num_crops] <= Inf
              0 <= w[i=1:num_crops] <= Inf
              0 <= e                <= Inf
              0 <= y[i=1:num_purch] <= Inf
           end
          );

#Define the objective
@objective(farmer_det,
            Max,
            sum(price[i]*w[i] for i=1:num_crops)+10e
           -sum(cost[i]*x[i]  for i=1:num_crops)
           -sum(extra_price[i]*y[i] for i=1:num_purch)
          );

#Add the constraints
@constraints(farmer_det,
             begin
                sum(x[i] for i=1:num_crops) <= land_size
                yield[1]*x[1]+y[1]-w[1]     == demand[1]
                yield[2]*x[2]+y[2]-w[2]     == demand[2]
                yield[3]*x[3]-w[3]-e        == 0
                w[3]                        <= over_size
             end
          );

In [4]:
#solve the model
optimize!(farmer_det)

#check the status 
status = termination_status(farmer_det);

if status != MOI.OPTIMAL
    println("Models status is: ", status, " === Oops! :/")
else
    println("Solved! Models status is: ", status, " === Yaay! :D")
    println("***********************************************")
    println("Optimal objective value = ", objective_value(farmer_det))   
    println("Acres of [wheat, corn, beans]  planted = ", value.(x));
    println("Tons of [wheat, corn, beans]  sold at regular price = ", value.(w));
    println("Tons of beans sold at extra price = ", value(e));
    println("Tons of [wheat, corn] purchased = ", value.(y));    
end

Solved! Models status is: OPTIMAL === Yaay! :D
***********************************************
Optimal objective value = 118600.0
Acres of [wheat, corn, beans]  planted = [120.0, 80.0, 300.0]
Tons of [wheat, corn, beans]  sold at regular price = [100.0, 0.0, 6000.0]
Tons of beans sold at extra price = 0.0
Tons of [wheat, corn] purchased = [0.0, 0.0]


### Stochastic Problem formulation

Suppose there are 3 possible **yield** outcomes (*scenarios*), each with **probability 1/3**:

|              |Wheat|Corn |Beans|
|--------------|-----|-----|-----|
|Good weather  |$3$  |$3.6$|$24$ |
|Normal weather|$2.5$|$3$  |$20$ |
|Bad weather   |$2$  |$2.4$|$16$ |



* **Decision variables**
    * First-stage decisions:
        * $x_1$: acres of wheat planted
        * $x_2$: acres of corn planted
        * $x_3$: acres of beans planted
        
    * Second-stage decisions (local copy for each scenario s):
        * $w_{1,s}$: tons of wheat sold at regular price, under scenario $s$
        * $w_{2,s}$: tons of corn sold at regular price, under scenario $s$
        * $w_{3,s}$: tons of beans sold at regular price, under scenario $s$
        * $e_{s}$: tons of beans sold at extra price, under scenario $s$
        * $y_{1,s}$: tons of wheat purchased, under scenario $s$
        * $y_{2,s}$: tons of corn purchased, under scenario $s$


In [5]:
#The stochastic yield
yield_stoch=[3 3.6 24;
            2.5 3 20;
            2 2.4 16]; 

#The number of scenarios
num_scens = size(yield_stoch)[1];

#The probabiliy of each scenario 
p = fill(1/num_scens,num_scens);

In [6]:
#Define the model
farmer_stoch = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GRB_ENV), "OutputFlag" => 0));

#Define the variables
@variables(farmer_stoch,
           begin
              0 <= xx[i=1:num_crops]               <= Inf
              0 <= ww[i=1:num_crops,s=1:num_scens] <= Inf
              0 <= ee[s=1:num_scens]               <= Inf
              0 <= yy[i=1:num_purch,s=1:num_scens] <= Inf
           end
          );


#Define the objective
@objective(farmer_stoch,
            Max,
            -sum(cost[i]*xx[i]  for i=1:num_crops)    
            +sum((
             sum(price[i]*ww[i,s] for i=1:num_crops)
            +10*ee[s]
            -sum(extra_price[i]*yy[i,s] for i=1:num_purch))*p[s]
            for s=1:num_scens)   
          );

#Add the constraints
@constraint(farmer_stoch, sum(xx[i] for i=1:num_crops) <= land_size);
for s=1:num_scens
    @constraints(farmer_stoch,
                 begin
                    yield_stoch[s,1]*xx[1]+yy[1,s]-ww[1,s] == demand[1]
                    yield_stoch[s,2]*xx[2]+yy[2,s]-ww[2,s] == demand[2]
                    yield_stoch[s,3]*xx[3]-ww[3,s]-ee[s]   == 0
                    ww[3,s]                                <= over_size
                 end
              );
end


In [7]:
#solve the model
optimize!(farmer_stoch)

#check the status 
status_stoch = termination_status(farmer_stoch);

if status_stoch != MOI.OPTIMAL
    println("Models status is: ", status_stoch, " === Oops! :/")
else
    println("Solved! Models status is: ", status_stoch, " === Yaay! :D")
    println("***********************************************")
    println("Optimal objective value = ", objective_value(farmer_stoch))   
    println("Acres of [wheat, corn, beans]  planted = ", value.(xx));
    for s=1:num_scens
        println("=========================================")
        println("in scenario s = ", s)
        println("Tons of [wheat, corn, beans]  sold at regular price = ", value.(ww)[:,s]);
        println("Tons of beans sold at extra price = ", value.(ee)[s]);
        println("Tons of [wheat, corn] purchased = ", value.(yy)[:,s]);
    end
end

Solved! Models status is: OPTIMAL === Yaay! :D
***********************************************
Optimal objective value = 108390.0
Acres of [wheat, corn, beans]  planted = [170.0, 80.0, 250.0]
in scenario s = 1
Tons of [wheat, corn, beans]  sold at regular price = [310.0, 48.0, 6000.0]
Tons of beans sold at extra price = 0.0
Tons of [wheat, corn] purchased = [0.0, 0.0]
in scenario s = 2
Tons of [wheat, corn, beans]  sold at regular price = [225.0, 0.0, 5000.0]
Tons of beans sold at extra price = 0.0
Tons of [wheat, corn] purchased = [0.0, 0.0]
in scenario s = 3
Tons of [wheat, corn, beans]  sold at regular price = [140.0, 0.0, 4000.0]
Tons of beans sold at extra price = 0.0
Tons of [wheat, corn] purchased = [0.0, 48.00000000000003]
